## 8) Using asynchronous functions with python 3

Pyrpl uses the Qt eventloop to perform asynchronous tasks, but it has been set as the default loop of `asyncio`, such that you only need to learn how to use the standard python module [`asyncio`](https://docs.python.org/3/library/asyncio.html), and you don't need to know anything about Qt. To give you a quick overview of what can be done, we present in the following block an exemple of 2 tasks running in parrallele. The first one mimicks a temperature control loop, measuring periodically a signal every 1 s, and changing the offset of an `asg` based on the measured value (we realize this way a slow and rudimentary software pid). In parrallele, another task consists in repeatedly shifting the frequency of an asg, and measuring an averaged spectrum on the spectrum analyzer.

Both tasks are defined by coroutines (a python function that is preceded by the keyword `async`, and that can contain the keyword `await`). Basically, the execution of each coroutine is interrupted whenever the keyword `await` is encountered, giving the chance to other tasks to be executed. It will only be resumed once the underlying coroutine's value becomes ready.

Finally to execute the cocroutines, it is not enough to call `my_coroutine()`, since we need to send the task to the event loop. For that, we use the function `ensure_future` from pyrpl.async_utils module. This function immediately returns an object that is not the result of the task (not the object that is behind `return` inside the coroutine), but rather a Future object, that can be used to retrieve the actual result once it is ready (this is done by calling `future.result()` latter on).

If you are executing the code inside the ipython notebook, then, this is all you have to do, since an event loop is already running in the back (a qt eventloop if you are using the option %pylab qt). Otherwise, you have to use one of the functions (`LOOP.run_forever()`, `LOOP.run_until_complete()`, or `LOOP.run_in_executor()`) to launch the eventloop.

In [ ]:
#define hostname
HOSTNAME = '192.169.1.100'

In [1]:
import sys
sys.path.append('C:/Users/michael.croquette/Documents/github/pyrpl')
import pyrpl

C:\Users\michael.croquette\AppData\Local\miniconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
from pyrpl import Pyrpl
p = Pyrpl(config='')
import asyncio

INFO:pyrpl.redpitaya:Please choose the hostname of your Red Pitaya in the hostname selector window!


In [3]:
#no-test # Will be tested only on branch python3-only
async def run_temperature_lock(setpoint=0.1):  # coroutines can receive arguments
    with p.asgs.pop("temperature") as asg: #  use the context manager "with" to 
    # make sure the asg will be freed after the acquisition
        asg.setup(frequency=0, amplitue=0, offset=0) #  Use the asg as a dummy 
        while IS_TEMP_LOCK_ACTIVE: #  The loop will run untill this flag is manually changed to False
                await asyncio.sleep(1) #  Give way to other coroutines for 1 s
                measured_temp = asg.offset #  Dummy "temperature" measurment
                asg.offset+= (setpoint - measured_temp)*0.1  #  feedback with an integral gain
                print("measured temp: ", measured_temp) #  print the measured value to see how the execution flow works
    
async def run_n_fits(n): #  a coroutine to launch n acquisitions
    sa = p.spectrumanalyzer
    with p.asgs.pop("fit_spectra") as asg: # use contextmanager again
        asg.setup(output_direct='out1',
                  trigger_source='immediately')
        freqs = []  #  variables stay available all along the coroutine's execution
        for i in range(n): #  The coroutine qill be executed several times on the await statement inside this loop
            asg.setup(frequency=1000*i) #  Move the asg frequency
            sa.setup(input=asg, avg=10, span=100e3, baseband=True) #  setup the sa for the acquisition
            spectrum = await sa.single_async() #  wait for 10 averages to be ready
            freq = sa.data_x[spectrum[0].argmax()] #  take the max of the spectrum
            freqs.append(freq) #  append it to the result
            print("measured peak frequency: ", freq) #  print to show how the execution goes
        return freqs #  Once the execution is over, the Future will be filled with the result...

from pyrpl.async_utils import ensure_future, sleep, wait
IS_TEMP_LOCK_ACTIVE = True

temp_future = ensure_future(run_temperature_lock(0.5)) # send temperature control task to the eventloop
fits_future = ensure_future(run_n_fits(50)) # send spectrum measurement task to the eventloop 
sleep(5)
IS_TEMP_LOCK_ACTIVE = False
print(wait(fits_future))

measured peak frequency:  59876.59096717834
measured peak frequency:  61016.064137220375
measured peak frequency:  54828.356951475136
measured peak frequency:  60118.73483657836
measured peak frequency:  42904.634028673165
measured peak frequency:  60629.56526875495
measured peak frequency:  59567.857533693306
measured peak frequency:  40776.561945676796
measured peak frequency:  58538.7460887432
measured peak frequency:  58410.689234733574
measured peak frequency:  59333.62990617751
measured peak frequency:  49455.09135723113
measured peak frequency:  38702.506572008126
measured peak frequency:  54507.516324520104
measured peak frequency:  60552.26549506187
measured peak frequency:  59611.164033412926
measured peak frequency:  56801.829487085335
measured peak frequency:  53237.6579940319
measured peak frequency:  58562.4948143959
measured peak frequency:  51935.20337343215
measured peak frequency:  52955.0015926361
measured peak frequency:  59350.8593738079
measured peak frequency:  5

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyError: 1

KeyError: 11

...

KeyError: 11

KeyError: 1

KeyError: 11

KeyError: 1

KeyboardInterrupt: 

In [4]:
%debug

> <ipython-input-3-54438531c6a7>(22)run_n_fits()
     20             sa.setup(input=asg, avg=10, span=100e3, baseband=True) #  setup the sa for the acquisition
     21             spectrum = await sa.single_async() #  wait for 10 averages to be ready
---> 22             freq = sa.data_x[spectrum.argmax()] #  take the max of the spectrum
     23             freqs.append(freq) #  append it to the result
     24             print("measured peak frequency: ", freq) #  print to show how the execution goes



ipdb>  sa.data_x


array([0.00000000e+00, 4.65661287e-01, 9.31322575e-01, ...,
       6.10342249e+04, 6.10346906e+04, 6.10351562e+04])


ipdb>  len(sa.data_x)


131073


...

ipdb>  spectrum.argmax()


261369


ipdb>  spectrum[0].argmax()


121102


ipdb>  exit()


In [ ]:
a=2